# Pressure vessel, exact solution

The aim of this notebook is to derive the exact solution to the problem of the *pressure vessel*.

We consider a tube (radius: $R$, wall thickness: $h$), subjected to an inner pressure $p_\mathrm{in}$ and an outer pressure $p_\mathrm{out}$. The tube is long enough to be in a 2D state (invariance along the $zz$ axis of revolution). For the sake of simplicity, we will assume that the tube is in plane strain equilibrium

**Question: ** is this a reasonable assumption?

Owing to the symmetries, the displacement is also rotation invariant about the axis of revolution. This suggests to work in cylindrical coordinates $(r, \theta, z)$. In this basis, the components of the displacement are functions of $r$ only

\begin{equation}
\mathbf u=u_r(r)\,\mathbf e_r+u_\theta(r)\,\mathbf e_\theta+u_z(r)\,\mathbf e_z.
\end{equation}

We will in fact seek a radial displacement

\begin{equation}
\mathbf u = u(r)\,\mathbf e_r.
\end{equation}

**Question:** what is going to tell us whether or not this assumption is valid?

## Initialization

We use the [SymPy](https://www.sympy.org/) module, which implements a whole computer algebra system in Python.

In [1]:
import sympy

In [2]:
sympy.init_printing(use_latex='mathjax')

In [3]:
from sympy.abc import n, x, y, r, E, R, A, B

In [4]:
μ, ν = sympy.symbols('mu nu')
λ = 2*μ*ν/(1-2*ν)

It will be convenient to introduce the following dimensionless quantity

\begin{equation}
\eta=\frac{h}{2R}
\end{equation}

In [5]:
η = sympy.Symbol('eta')
h = 2*R*η

As well as the mean pressure $p$ and pressure difference $\Delta p$

\begin{equation}
p = \frac{p_\mathrm{in}+p_\mathrm{out}}{2}
\quad\text{and}\quad
\varpi = \frac{p_\mathrm{in}-p_\mathrm{out}}{p},
\end{equation}

such that

\begin{equation}
p_\mathrm{in} = p\bigl(1+\frac{\varpi}{2}\bigr)
\quad\text{and}\quad
p_\mathrm{out} = p\bigl(1-\frac{\varpi}{2}\bigr).
\end{equation}

In [6]:
from sympy.abc import p
ϖ = sympy.Symbol('varpi')
p_in = p*(1+ϖ/2)
p_out = p*(1-ϖ/2)

## General form of the radial displacement

In [7]:
u = sympy.Function('u')(r)

The only non-vanishing components of the strain tensor $\boldsymbol\varepsilon$ are

\begin{equation}
\varepsilon_{rr}=u',\quad\text{and}\quad\varepsilon_{\theta\theta}=\frac ur,
\end{equation}

where $'$ refers to derivation with respect to $r$.

In [8]:
ε_rr = u.diff(r)
ε_θθ = u/r
tr_ε = ε_rr + ε_θθ

The stresses $\boldsymbol\sigma$ are deduced from Hooke's law (plane strain).

In [9]:
σ_rr = λ*tr_ε + 2*μ*ε_rr
σ_θθ = λ*tr_ε + 2*μ*ε_θθ

The equilibrium equation reduces to

\begin{equation}
\sigma_{rr}'+\frac{\sigma_{rr}-\sigma_{\theta\theta}}r=0
\end{equation}

In [10]:
lhs = σ_rr.diff(r) + (σ_rr-σ_θθ)/r
eq1 = sympy.Equality(lhs.expand(), 0)
eq1

                                           2                                  
                                          d                                   
                        d          2⋅μ⋅ν⋅───(u(r))                       d    
                  2⋅μ⋅ν⋅──(u(r))           2               2         2⋅μ⋅──(u(
    2⋅μ⋅ν⋅u(r)          dr               dr               d              dr   
- ───────────── + ────────────── + ─────────────── + 2⋅μ⋅───(u(r)) + ─────────
         2    2     -2⋅ν⋅r + r         -2⋅ν + 1            2              r   
  - 2⋅ν⋅r  + r                                           dr                   

                  
                  
                  
r))               
      2⋅μ⋅u(r)    
─── - ──────── = 0
          2       
         r        

We seek a solution of the form $u(r)=r^n$.

In [11]:
eq2 = eq1.subs(u, r**n).doit()
eq2.factor()

     n                            
2⋅μ⋅r ⋅(n - 1)⋅(n + 1)⋅(ν - 1)    
────────────────────────────── = 0
          2                       
         r ⋅(2⋅ν - 1)             

Which results in $n=\pm 1$. In other words

\begin{equation}
u(r) = Ar+\frac{B}r,
\end{equation}

where $A$ and $B$ are two integration constants which are found from the boundary conditions at $r=R\pm h/2$.

## Determination of the integration constants

In [12]:
def subs_u_sol(expr):
    return expr.subs(u, u_sol).doit().expand().collect(r, func=sympy.factor)

In [13]:
u_sol = A*r+B/r

In [14]:
ε_rr_sol = subs_u_sol(ε_rr)
ε_θθ_sol = subs_u_sol(ε_θθ)
σ_rr_sol = subs_u_sol(σ_rr)
σ_θθ_sol = subs_u_sol(σ_θθ)

In [15]:
σ_rr_sol

   2⋅A⋅μ    2⋅B⋅μ
- ─────── - ─────
  2⋅ν - 1      2 
              r  

At $r=R-h/2$, the boundary condition reads $\sigma_{rr}=-p_\mathrm{in}$.

In [16]:
eq3 = sympy.Equality(σ_rr_sol.subs(r, R-h/2), -p_in)

At $r=R+h/2$, the boundary condition reads $\sigma_{rr}=-p_\mathrm{out}$.

In [17]:
eq4 = sympy.Equality(σ_rr_sol.subs(r, R+h/2), -p_out)

The integration constants follow.

In [18]:
sol = sympy.solve([eq3, eq4], [A, B])
sol

⎧      ⎛   2            2                                        ⎞       2    
⎪   -p⋅⎝2⋅η ⋅ν⋅varpi - η ⋅varpi - 8⋅η⋅ν + 4⋅η + 2⋅ν⋅varpi - varpi⎠      R ⋅p⋅v
⎨A: ───────────────────────────────────────────────────────────────, B: ──────
⎪                                8⋅η⋅μ                                        
⎩                                                                             

     ⎛ 4      2    ⎞⎫
arpi⋅⎝η  - 2⋅η  + 1⎠⎪
────────────────────⎬
    8⋅η⋅μ           ⎪
                    ⎭

In [19]:
A_sol = sol[A].expand().collect(ϖ, func=sympy.factor)
sympy.Equality(A, A_sol)

                          ⎛ 2    ⎞          
    p⋅(2⋅ν - 1)   p⋅varpi⋅⎝η  + 1⎠⋅(2⋅ν - 1)
A = ─────────── - ──────────────────────────
        2⋅μ                 8⋅η⋅μ           

In [20]:
B_sol = sol[B].factor()
sympy.Equality(B, B_sol)

     2                2        2
    R ⋅p⋅varpi⋅(η - 1) ⋅(η + 1) 
B = ────────────────────────────
               8⋅η⋅μ            